In [1]:
import numpy as np 
import pandas as pd


In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer


In [57]:
df = pd.read_csv(r'C:\Users\Lenovo\Krishnaraj singh\Code\newml\Documents!.0\covid_toy.csv')
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [58]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2,random_state=0)

In [59]:
x_train

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


In [60]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [61]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

## __Without colm tranformer__

- Fill the missing value of fever by its mean

In [62]:
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.fit_transform(x_test[['fever']])
u = x_train['fever'].isnull().sum()
print(u)

x_train_fever = pd.DataFrame(x_train_fever,columns=['fever'])
x_train_fever.isnull().sum()

x_train_fever.shape


9


(80, 1)

- ordinary encoding

In [63]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.fit_transform(x_test[['cough']])

x_test_cough.shape

(20, 1)

## __One hot encoding__

In [64]:
ohe = OneHotEncoder(drop='first', sparse_output=False)  #,sparse= it is removed in new version
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

x_test_gender_city = ohe.transform(x_test[['gender','city']])

x_train_gender_city
x_train_gender_city.shape

(80, 4)

Extract age

In [65]:
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

Sabko add krege 

In [ ]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)
x_train_transformed.shape


(80, 7)

In [71]:
# Column names
age_cols = ['age']
fever_cols = ['fever']
ohe_cols = ohe.get_feature_names_out(['gender','city']).tolist()
cough_cols = ['cough']

# Combine
all_cols = age_cols + fever_cols + ohe_cols + cough_cols

# Convert to DataFrame
x_train_df = pd.DataFrame(x_train_transformed, columns=all_cols)
x_test_df = pd.DataFrame(x_test_transformed, columns=all_cols)

x_train_df


,age,fever,gender_Male,city_Delhi,city_Kolkata,city_Mumbai,cough
0,22.0,99.0,0.0,0.0,0.0,0.0,0.0
1,56.0,104.0,0.0,0.0,0.0,0.0,1.0
2,31.0,98.0,0.0,0.0,1.0,0.0,0.0
3,75.0,104.0,0.0,1.0,0.0,0.0,1.0
4,72.0,99.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
75,51.0,101.0,0.0,0.0,1.0,0.0,1.0
76,65.0,99.0,1.0,0.0,0.0,0.0,0.0
77,42.0,104.0,1.0,0.0,0.0,1.0,0.0
78,18.0,104.0,0.0,0.0,0.0,0.0,0.0


## __Colmn transformer__

In [79]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [81]:
transformer.fit_transform(x_train).shape

(80, 7)